# File Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy
import datetime
import pyproj
import rioxarray
from scipy.interpolate import griddata
import scipy as sp
import folium
import os
import folium
from ipywidgets import interact, Dropdown
from IPython.display import display, clear_output
from itertools import combinations
import matplotlib
import dask.array as da
from concurrent.futures import ThreadPoolExecutor
from scipy.sparse import csr_matrix
from oceanum.datamesh import Connector
import netCDF4 as nc
datamesh=Connector(token='3052e2bdd10904ae353ac54ed205df32bfcc20e2')
import dask

In [2]:
# Define the current and target CRS using EPSG codes
current_crs = pyproj.CRS("EPSG:2193")  
target_crs = pyproj.CRS("EPSG:4326")

transformer = pyproj.Transformer.from_crs(current_crs, target_crs, always_xy=True,allow_ballpark=False)

# Load DEM

In [3]:
xr_dem = rioxarray.open_rasterio('dem_8m_2012_wgs//92L94-92L34.tif') #datamesh.query({'datasource':'linz-wellington_2013-2014-dem_1m-2193',"geofilter":{"type":"bbox","geom":[174.536322,-41.442487,175.124777,-41.049083]}})
# xr_dem = xr_dem.band_data

In [4]:
df_vlms = pd.read_csv('our-changing-coast-insar_rates_searise.csv')

In [5]:
dem_values = xr_dem.values
fill_value = np.min(dem_values)

dem_values[dem_values==fill_value] = np.nan
xr_dem.values = dem_values

In [6]:
# Create a regular grid of coordinates
res = 4000# low res to make the initial calcs fast
x_regular = np.linspace(xr_dem['x'].min(), xr_dem['x'].max(), res)
y_regular = np.linspace(xr_dem['y'].min(), xr_dem['y'].max(), res)

# Interpolate data onto regular grid
data_interpolated = xr_dem[0,:,:].interp(y=y_regular, x=x_regular)

# Now you can use the interpolated data for transformation
new_DEM = xr.DataArray(data_interpolated,coords={"y":y_regular,"x":x_regular},dims=["y","x"])

In [7]:
new_DEM = new_DEM.chunk(100)

# Load Seismic Data

In [8]:
array_2pc = np.load('for_DataMesh/for_DataMesh/2perc_disps_sites_c_MDEz_uniform.npy')
array_10pc = np.load('for_DataMesh/for_DataMesh/10perc_disps_sites_c_MDEz_uniform.npy')

In [9]:
df_2pc = pd.DataFrame(array_2pc)
df_2pc.columns = ['ID','Lon','Lat','Uplift','Subsidence','absVLM']
lon_lats = [transformer.transform(x,y) for x,y in zip(df_2pc.Lon,df_2pc.Lat)]
df_2pc['Lon'] = [x[0] for x in lon_lats]
df_2pc['Lat'] = [x[1] for x in lon_lats]

df_10pc = pd.DataFrame(array_10pc)
df_10pc.columns = ['ID','Lon','Lat','Uplift','Subsidence','absVLM']
lon_lats = [transformer.transform(x,y) for x,y in zip(df_10pc.Lon,df_10pc.Lat)]
df_10pc['Lon'] = [x[0] for x in lon_lats]
df_10pc['Lat'] = [x[1] for x in lon_lats]

In [10]:
# Put onto DEM grid
x = new_DEM.x
y = new_DEM.y
X, Y = np.meshgrid(x, y)

array_2pc = griddata((df_2pc['Lon'], df_2pc['Lat']), df_2pc.absVLM, (X, Y), method='linear')
array_10pc = griddata((df_10pc['Lon'], df_10pc['Lat']), df_10pc.absVLM, (X, Y), method='linear')
array_none = array_10pc.copy()
array_none[:,:] = 0

In [11]:
array_2pc = array_2pc.astype(np.float32)
array_2pc = csr_matrix(array_2pc)

array_10pc = array_10pc.astype(np.float32)
array_10pc = csr_matrix(array_10pc)

array_none = array_none.astype(np.float32)
array_none = csr_matrix(array_none)

In [12]:

earthquake_scens = {
    'None':array_none,
    '2pc':array_2pc,
    '10pc':array_10pc
}

# Storm Surge

DataSet: https://data.4tu.nl/articles/_/13392314/1

In [13]:
xr_storm_surge = xr.open_dataset('COAST-RP.nc')
xr_100rp = xr_storm_surge['storm_tide_rp_0100']

In [14]:
well_y = -41.274678
well_x = 174.854143

In [15]:
df_storm_surge = xr_storm_surge.to_dataframe()

In [16]:
df_storm_surge.loc[:,'euclidean'] = (df_storm_surge.station_x_coordinate-well_x)**2+(df_storm_surge.station_y_coordinate-well_y)**2
df_storm_surge = df_storm_surge.sort_values('euclidean').reset_index(drop=True)
storm_tide_rps_dict = df_storm_surge.loc[0,[x for x in df_storm_surge.columns if 'storm_tide_rp' in x]].to_dict()

In [17]:
df_storm_surge

,station_id,storm_tide_rp_0001,storm_tide_rp_0002,storm_tide_rp_0005,storm_tide_rp_0010,storm_tide_rp_0025,storm_tide_rp_0050,storm_tide_rp_0100,storm_tide_rp_0250,storm_tide_rp_0500,storm_tide_rp_1000,station_x_coordinate,station_y_coordinate,euclidean
0,islands_27003,0.536,0.564,0.598,0.620,0.648,0.668,0.687,0.704,0.712,0.722,174.858002,-41.264999,0.000109
1,id_coast_glob_08580,0.534,0.562,0.596,0.619,0.646,0.667,0.686,0.703,0.712,0.720,174.800003,-41.293999,0.003304
2,id_coast_glob_08581,1.069,1.111,1.160,1.192,1.234,1.264,1.294,1.329,1.354,1.378,174.770996,-41.146999,0.023215
3,islands_26834,1.134,1.178,1.228,1.262,1.302,1.333,1.365,1.403,1.431,1.454,174.770996,-41.089001,0.041389
4,id_coast_glob_08579,0.996,1.027,1.064,1.089,1.119,1.141,1.159,1.179,1.193,1.207,175.181000,-41.439999,0.134168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23221,id_coast_glob_08194,2.208,2.342,2.528,2.668,2.840,2.974,3.110,3.295,3.397,3.465,-179.735992,66.108002,137265.218750
23222,id_coast_glob_08197,0.733,0.794,0.875,0.931,0.997,1.036,1.068,1.109,1.129,1.146,-178.212997,71.528000,137380.843750
23223,id_coast_glob_08198,0.846,0.903,0.978,1.027,1.082,1.123,1.161,1.200,1.217,1.233,-178.856995,70.913002,137697.625000
23224,id_coast_glob_08195,1.096,1.207,1.329,1.400,1.479,1.540,1.618,1.679,1.706,1.724,-179.854004,68.920998,137960.953125


In [18]:
storm_tide_rps_dict = {k.split('_')[-1]:v for k,v in storm_tide_rps_dict.items()}

In [19]:
storm_tide_rps_dict

{'0001': 0.5360000133514404,
 '0002': 0.5640000104904175,
 '0005': 0.5979999899864197,
 '0010': 0.6200000047683716,
 '0025': 0.6480000019073486,
 '0050': 0.6679999828338623,
 '0100': 0.6869999766349792,
 '0250': 0.7039999961853027,
 '0500': 0.7120000123977661,
 '1000': 0.722000002861023}

# Load VLMs

In [20]:
df_vlms = pd.read_csv('Welly_VLM_2018-2023_100m.txt',delimiter='\t')
df_vlms = df_vlms.astype(np.float32)

In [21]:
df_vlms = df_vlms.rename(columns={'  0.000000':'lon','  0.000000.1':'lat'})

In [22]:
df_vlms.set_index(['lat','lon'],inplace=True)
df_vlms.columns = df_vlms.columns.astype(float)
df_vlms = df_vlms.reset_index()

In [23]:
cols = [x for x in df_vlms.columns if x not in ['lon','lat']]

In [24]:
# Precompute grid once
x = new_DEM.x.astype(np.float32)
y = new_DEM.y.astype(np.float32)

X, Y = np.meshgrid(x, y)

# Function to perform interpolation for a given year
def interpolate_year(year):
    Z_year = griddata((df_vlms['lon'].astype(np.float32), df_vlms['lat'].astype(np.float32)), df_vlms[year].astype(np.float32), (X, Y), method='linear')
    return np.nan_to_num(Z_year, nan=0).astype(np.float32)

# Function to process a chunk of years
def process_chunk(chunk):
    with ThreadPoolExecutor() as executor:
        return list(executor.map(interpolate_year, chunk))

# Determine chunk size and number of chunks
chunk_size = 10  # Adjust based on available memory
chunks = [cols[i:i + chunk_size] for i in range(0, len(cols), chunk_size)]

# Initialize list to collect results
all_chunks_results = []

# Process each chunk
for chunk in chunks:
    chunk_results = process_chunk(chunk)
    all_chunks_results.append(np.stack(chunk_results, axis=-1))

# Concatenate all chunks along the time dimension
duplicated_arr = np.concatenate(all_chunks_results, axis=-1)

# Create DataArray
duplicated_arr = xr.DataArray(duplicated_arr, coords={"lon": np.array(x), "lat": np.array(y), "years": cols}, dims=['lon', 'lat', 'years'])

# Interpolate NaN values
duplicated_arr = (duplicated_arr.chunk(dict(lat=-1)).interpolate_na(dim='lat', method='linear') + duplicated_arr.chunk(dict(lon=-1)).interpolate_na(dim='lon', method='linear')) / 2

# rechunk
duplicated_arr = duplicated_arr.chunk(100)

# Load SLR Data

In [25]:
xr_slr = sp.io.loadmat('Ian_relabelled_sites/total_volc_noVLMssp585_medium_confidence_values.mat')

lats = {x:y for x,y in zip(xr_slr['locations'].squeeze(),xr_slr['lat'].squeeze())}
lons = {x:y for x,y in zip(xr_slr['locations'].squeeze(),xr_slr['lon'].squeeze())}

x = new_DEM.x
y = new_DEM.y

years = np.unique(xr_slr['years'])
quantiles = [0.17,0.5,0.83]

X, Y = np.meshgrid(x, y)

example_year = np.min(years)
example_quantile = np.min(quantiles)

In [26]:
file_names = os.listdir("Ian_relabelled_sites/")
file_names = [x for x in file_names if ('total' in x)&('medium' in x)&('_noVLM' in x)]

In [ ]:
# Function to process each file
def process_file(k, file):
    print('testing')
    xr_slr = sp.io.loadmat(f'Ian_relabelled_sites/{file}')
    
    xr_slr = xr.DataArray(xr_slr['sea_level_change'],
                          coords={'locations': xr_slr['locations'].squeeze(),
                                  'years': xr_slr['years'].squeeze(),
                                  'quantiles': xr_slr['quantiles'].squeeze()},
                          dims=['locations', 'years', 'quantiles'])
    
    df_slr = xr_slr.sel(quantiles=quantiles).to_dataframe('slr')
    
    del xr_slr
    
    df_slr['Lat'] = [lats[x] for x in df_slr.reset_index()['locations']]
    df_slr['Lon'] = [lons[x] for x in df_slr.reset_index()['locations']]
    
    # Focus on the wellington region
    df_slr = df_slr[(df_slr.Lon > np.min(new_DEM.x.values)) & 
                    (df_slr.Lat > np.min(new_DEM.y.values)) & 
                    (df_slr.Lon < np.max(new_DEM.x.values)) & 
                    (df_slr.Lat < np.max(new_DEM.y.values))]
    
    df_slr = df_slr.reset_index().drop('locations', axis=1)

    slr_array = da.zeros((len(x), len(y), len(years), len(quantiles), len(file_names)), dtype=np.float32, chunks=10)
    
    for i, year in enumerate(years):
        for j, quantile in enumerate(quantiles):
            df_slr_inst = df_slr[(df_slr.years == year) & (df_slr.quantiles == quantile)]
            if not df_slr_inst.empty:
                grid = griddata((df_slr_inst['Lon'].astype(np.float32), df_slr_inst['Lat'].astype(np.float32)), 
                                                              df_slr_inst['slr'].astype(np.float32), (X, Y), 
                                                              method='linear').astype(np.float32)
                                                    # da.from_array( chunks=1000)
                slr_array[:,:,i,j,k] = grid
                
            print(i,j)

    return(k)

# Use ThreadPoolExecutor to process files in parallel
with ThreadPoolExecutor() as executor:
    results = list(executor.map(process_file, range(len(file_names)), file_names))

# Rechunk
slr_array = slr_array.rechunk(100)

# Finalize the Dask array
# slr_array = slr_array.compute()

# Create an xarray DataArray
xr_slr = xr.DataArray(slr_array, coords={"Lon": np.array(x), "Lat": np.array(y), "years": years, "quantiles": quantiles, "scenarios": file_names},
                      dims=['Lon', 'Lat', 'years', 'quantiles', 'scenarios'])

# Fill missing values
filled_x = xr_slr.ffill('Lon').bfill('Lat')
filled_y = xr_slr.ffill('Lat').bfill('Lon')
xr_slr_filled = (filled_x + filled_y) / 2


testing
testing
testing
testing
testing


In [ ]:
asdf

In [ ]:
# k = 0
# for file in file_names:
#     xr_slr = sp.io.loadmat(f'../../Ian_relabelled_sites/{file}')
    
#     xr_slr = xr.DataArray(xr_slr['sea_level_change'],
#                           coords={'locations':xr_slr['locations'].squeeze(),'years':xr_slr['years'].squeeze(),'quantiles':xr_slr['quantiles'].squeeze()},
#                          dims=['locations','years','quantiles'])
    
#     df_slr = xr_slr.sel(quantiles=quantiles).to_dataframe('slr')
    
#     df_slr['Lat'] = [lats[x] for x in df_slr.reset_index()['locations']]
#     df_slr['Lon'] = [lons[x] for x in df_slr.reset_index()['locations']]
    
#     # Focus on the wellington region
#     df_slr = df_slr[(df_slr.Lon>np.min(new_DEM.x.values))&(df_slr.Lat>np.min(new_DEM.y.values))&(df_slr.Lon<np.max(new_DEM.x.values))&(df_slr.Lat<np.max(new_DEM.y.values))]
    
#     df_slr = df_slr.reset_index().drop('locations',axis=1)
    
#     i = 0
#     for year in years:
#         j = 0
#         for quantile in quantiles:
#             df_slr_inst = df_slr[(df_slr.years==year)&(df_slr.quantiles==quantile)]
#             slr_array[:,:,i,j,k] = griddata((df_slr_inst['Lon'], df_slr_inst['Lat']), df_slr_inst['slr'], (X, Y), method='linear')
#             j+=1
#         i+=1
#     k+=1

# xr_slr = xr.DataArray(slr_array,coords={"Lon":np.array(x),"Lat":np.array(y),"years":years,"quantiles":quantiles,"scenarios":file_names},dims=['Lon','Lat','years','quantiles','scenarios'])

# filled_x = xr_slr.ffill('Lon').bfill('Lat')
# filled_y = xr_slr.ffill('Lat').bfill('Lon')
# xr_slr_filled = (filled_x+filled_y)/2
    

# Adding SLR and VLM together Data

In [ ]:
# Find a mean of the rates and project forward
xr_vlm_mean = duplicated_arr.mean(dim='years',skipna=True)
xr_vlm_mean = xr_vlm_mean.expand_dims({'years':xr_slr.years.values})

year_count = (xr_vlm_mean.years-np.min(xr_vlm_mean.years)) #should really start when the DEM was made for
xr_vlm_forecast = xr_vlm_mean*np.array(year_count)[:,None,None]
xr_vlm_forecast = xr_vlm_forecast.transpose('lon','lat','years')
xr_slr_vlm_adjusted = xr_slr_filled-np.array(xr_vlm_forecast)[:,:,:,None,None]

# Flooding the DEM

In [ ]:
new_DEM_mask = new_DEM.copy()
new_DEM_mask = new_DEM_mask/new_DEM_mask
new_DEM_mask = new_DEM_mask.fillna(123456789)#where(new_DEM_mask!=float('NaN'),123456789)
new_DEM_mask = new_DEM_mask.where(new_DEM_mask!=1,np.nan)
new_DEM_mask = new_DEM_mask.where(new_DEM_mask!=123456789,1)

In [ ]:
(np.empty((len(list(storm_tide_rps_dict.keys())),
         len(list(earthquake_scens.keys())),
         len(years),
         len(file_names),
         len(x),
         len(y))).astype(np.float32)).shape

In [ ]:
masked_xarray = xr.DataArray(np.empty((len(list(storm_tide_rps_dict.keys())),
         len(list(earthquake_scens.keys())),
         len(years),
         len(file_names),
         len(x),
         len(y))).astype(np.float32),dims=['stormsurge','earthquake','years','file','lon','lat'],
                            coords=[list(storm_tide_rps_dict.keys()),list(earthquake_scens.keys()),years,file_names,x,y])

flooded_xarray = xr.DataArray(np.empty((len(list(storm_tide_rps_dict.keys())),
         len(list(earthquake_scens.keys())),
         len(years),
         len(file_names),
         len(x),
         len(y))).astype(np.float32),dims=['stormsurge','earthquake','years','file','lon','lat'],
                            coords=[list(storm_tide_rps_dict.keys()),list(earthquake_scens.keys()),years,file_names,x,y])

In [ ]:
masked_array_dict = {}
flooded_dict = {}

for storm_rps,storm_value in storm_tide_rps_dict.items():
    for e_scen,e_array in earthquake_scens.items():
        for year in years:
            for scenario in file_names:
                xr_slr_vlm_adjusted_year = xr_slr_vlm_adjusted.sel(years=year,scenarios=scenario)
                
                new_DEM_flooded_low = new_DEM-np.array(xr_slr_vlm_adjusted_year.sel(quantiles=np.min(quantiles)))/1000-e_array-float(storm_value)
                new_DEM_flooded_low = new_DEM_flooded_low
                flooded_low = new_DEM_flooded_low.where(new_DEM_flooded_low<0,np.nan)*0+1
                
                new_DEM_flooded_mid = new_DEM-np.array(xr_slr_vlm_adjusted_year.sel(quantiles=0.5))/1000-e_array-float(storm_value)
                new_DEM_flooded_mid = new_DEM_flooded_mid
                flooded_mid = new_DEM_flooded_mid.where(new_DEM_flooded_mid<0,np.nan)*0+1
                
                new_DEM_flooded_high = new_DEM-np.array(xr_slr_vlm_adjusted_year.sel(quantiles=np.max(quantiles)))/1000-e_array-float(storm_value)
                new_DEM_flooded_high = new_DEM_flooded_high
                flooded_high = new_DEM_flooded_high.where(new_DEM_flooded_high<0,np.nan)*0+1
                
                flooded = flooded_low.fillna(0)+flooded_mid.fillna(0)+flooded_high.fillna(0)
                flooded = flooded.where(flooded>0,np.nan)
                
                masked_flooded = np.ma.masked_invalid(flooded)
                masked_flooded = (np.max(masked_flooded)-masked_flooded)/(np.max(masked_flooded)-np.min(masked_flooded))
                masked_flooded = masked_flooded[::-1,:]
        
                masked_array_dict.update({
                    (year,scenario,e_scen,storm_rps):masked_flooded
                })
        
                flooded_dict.update({
                    (year,scenario,e_scen,storm_rps):flooded
                })

                masked_xarray.loc[{'stormsurge': storm_rps, 'earthquake': e_scen, 'years': year, 'file': scenario}] = masked_flooded
                flooded_xarray.loc[{'stormsurge': storm_rps, 'earthquake': e_scen, 'years': year, 'file': scenario}] = flooded

     

In [ ]:
xr_slr_vlm_adjusted_year = xr_slr_vlm_adjusted.sel(years=year,scenarios=scenario)

new_DEM_flooded_low = new_DEM-np.array(xr_slr_vlm_adjusted_year.sel(quantiles=np.min(quantiles)))/1000-e_array-float(storm_value)
new_DEM_flooded_low = new_DEM_flooded_low
flooded_low = new_DEM_flooded_low.where(new_DEM_flooded_low<0,np.nan)*0+1

new_DEM_flooded_mid = new_DEM-np.array(xr_slr_vlm_adjusted_year.sel(quantiles=0.5))/1000-e_array-float(storm_value)
new_DEM_flooded_mid = new_DEM_flooded_mid
flooded_mid = new_DEM_flooded_mid.where(new_DEM_flooded_mid<0,np.nan)*0+1

new_DEM_flooded_high = new_DEM-np.array(xr_slr_vlm_adjusted_year.sel(quantiles=np.max(quantiles)))/1000-e_array-float(storm_value)
new_DEM_flooded_high = new_DEM_flooded_high
flooded_high = new_DEM_flooded_high.where(new_DEM_flooded_high<0,np.nan)*0+1

flooded = flooded_low.fillna(0)+flooded_mid.fillna(0)+flooded_high.fillna(0)
flooded = flooded.where(flooded>0,np.nan)

masked_flooded = np.ma.masked_invalid(flooded)
masked_flooded = (np.max(masked_flooded)-masked_flooded)/(np.max(masked_flooded)-np.min(masked_flooded))
masked_flooded = masked_flooded[::-1,:]

masked_array_dict.update({
    (year,scenario,e_scen,storm_rps):masked_flooded
})

flooded_dict.update({
    (year,scenario,e_scen,storm_rps):flooded
})

masked_xarray.loc[{'stormsurge': storm_rps, 'earthquake': e_scen, 'years': year, 'file': scenario}] = masked_flooded
flooded_xarray.loc[{'stormsurge': storm_rps, 'earthquake': e_scen, 'years': year, 'file': scenario}] = flooded

# Save to the mesh

In [ ]:
data_vars = {'mask': masked_xarray, 'data': flooded_xarray}
# Create a Dataset from the dictionary
xr_flooded = xr.Dataset(data_vars)

In [ ]:
xr_flooded

In [ ]:
# datamesh.write_datasource(datasource_id="wellington_dynamic_shoreline_projections_mask",
#                           name="Wellington Casestudy Dynamic Shoreline Projections Mask",
#                           description="Wellington Casestudy Dynamic Shoreline Projections Mask",
#                           data=xr_flooded,
#                           # coordinates={'lon':'longitude','lat':'latitude'},
#                          tags=['demo', 'wellington','lower hutt'],
#                          )

# Visualise

In [ ]:
cmap = plt.cm.get_cmap('brg')

wellington_coords = [-41.28664, 174.77557]
zoom = 11

def create_map(year,ssp, earthquake,storm_surge):
    token = "pk.eyJ1Ijoic2hhbm5vbi1iZW5ndHNvbiIsImEiOiJja3F1Y2Q0dHEwMzYwMm9wYmtzYzk2bDZuIn0.5jGMyEiJdmXs1HL7x3ThPw" # your mapbox token
    tileurl = 'https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=' + str(token)
    m = folium.Map(location=wellington_coords, zoom_start=zoom)
    custom_tile_layer = folium.TileLayer(tiles=tileurl, name='Satellite',attr='Mapbox',overlay=False).add_to(m)

    masked_flooded = masked_array_dict[(year,ssp,earthquake,storm_surge)]
    flooded = flooded_dict[(year,ssp,earthquake,storm_surge)]

    # print(year,ssp)
    
    folium.raster_layers.ImageOverlay(cmap(masked_flooded),
                                      [[flooded.y.values.min(), flooded.x.values.min()],
                                       [flooded.y.values.max(), flooded.x.values.max()]],opacity=.5).add_to(m)

    return(m)
    
# Define dropdown options
dropdown_options1 = ['ssp126','ssp126', 'ssp245','ssp370','ssp585']
dropdown_options2 = years
dropdown_options3 = list(earthquake_scens.keys())
dropdown_options4 = list(storm_tide_rps_dict.keys())

# Define callback function to update map when dropdown value changes
def update_map(year, ssp, earthquake,storm_surge):
    clear_output(wait=True)
    try:
        m = create_map(year, f'total_volc_noVLM{ssp}_medium_confidence_values.mat',earthquake,storm_surge)
        display(m)
    except Exception as e:
        print(f"Error occurred: {e}")

# Create interactive dropdowns
interact(update_map, year=dropdown_options2, ssp=dropdown_options1,earthquake=dropdown_options3,storm_surge=dropdown_options4)


In [ ]:
masked_flooded.shape

In [ ]:
array_10pc.shape